In [ ]:
pip install requests

In [ ]:
pip install pandas

In [ ]:
pip install time

In [9]:
import requests

import pandas as pd

import time

client_id = "45da58455f7f46d5b30b0b8a117fa452"
client_secret = "7dd2efbeaecd4fd08136e682f7610f48"


# 獲取 access_token
def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))

    if response.status_code == 200:
        return response.json().get('access_token')
    else:
        print(f"Failed to obtain access token. Status Code: {response.status_code}, Response: {response.text}")
        return None


# 查詢專輯
def get_album_track(album_id, access_token):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks?market=TW&limit=20"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    while True:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # 取得專輯歌曲
            tracks = data.get('items', [])
            album_tracks = []
            for track in tracks:
                track_id = track.get("id")  # 取得歌曲 ID
                track_name = track.get("name")  # 取得歌曲 名稱
                track_ms = track.get("duration_ms") # 取得歌曲 長度
                track_number = track.get("track_number") # 取得歌曲 編號
                album_tracks.append((track_id, track_name, track_ms, track_number))  # 將 專輯 info 添加到列表中
            return album_tracks
        
        elif response.status_code == 401:  # Unauthorized, possibly due to an expired token
            print("Access token expired. Refreshing token...")
            access_token = get_access_token()  # Get a new token
            return get_album_track(album_id, access_token)  # Retry with new token
        
        elif response.status_code == 429:  # Too Many Requests
            retry_after = int(response.headers.get("Retry-After", 1))
            print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            print(f"Failed to get tracks for album {album_id}. Status Code: {response.status_code}, Response: {response.text}")
            return []


# 主程式
def main():
    access_token = get_access_token()

    if not access_token:
        return

    # 讀取 CSV
    artist_album_ids_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify_va13\artist_album_ids.csv"
    df = pd.read_csv(artist_album_ids_path)

    # 將查詢結果存入 DataFrame
    result_data = []
    
    for index, row in df.iterrows():
        album_id = row["Artist_album_id"]

        try:
            album_track_info = get_album_track(album_id, access_token)

            if album_track_info:
                for track_id, track_name, track_ms, track_number in album_track_info:  # 正確解包專輯 info
                    result_data.append({"Artist_album_id": album_id, "Track_id": track_id, "Track_name": track_name, "Track_ms": track_ms, "Track_number": track_number})
            else:
                print(f"Album ID {album_id} has no tracks.")
        
        except Exception as e:
            print(f"Error processing album {album_id}: {e}")
    
        # 添加延遲，防止達到速率限制
        time.sleep(1)  # 每次請求後等待 1 秒
            
    result_df = pd.DataFrame(result_data)

    # 保存結果到 CSV
    result_csv_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify_va13\album_tracks.csv"
    result_df.to_csv(result_csv_path, index=False)
    

if __name__ == "__main__":
    main()

Rate limit exceeded. Retrying after 38630 seconds...
